In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
working_dir = "/content/drive/Shareddrives/ML/AIIJC/working_dir"#@param{type:"string"}

In [15]:
#@markdown Очистка `working_dir`

import os, shutil
folder = working_dir
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# Подключаем библиотеки и модельки

In [3]:
#@title Подключаем библиотеки GPT-Neo (30 сек)
#@markdown ### и модель
%%capture

!pip install happytransformer

from happytransformer import HappyGeneration
from happytransformer import GENSettings

import re

model_path = "/content/drive/Shareddrives/ML/AIIJC/2.1 GPT-Neo/poems" #@param {type:"string"}

happy_gen = HappyGeneration(load_path = model_path)

In [4]:
#@title Подключаем библиотеки Silero

%%capture

!pip install torch
!pip install torchaudio
!pip install -q torchaudio omegaconf

import torch

In [5]:
# @title Генеральные функции
from IPython.display import display, Audio
from scipy.io.wavfile import write

def displayAudio(audio, rate = 16000):
  display(Audio(audio, rate=rate))

def createAudioFile(audio, name, rate = 16000):
  write(f'{name[:60]}.wav', rate, audio)

def generateRawAudio(speakerLines, lines, language = 'en'):
  wavs = []
  for i in range(len(lines)):
    speaker = speakerLines[i]
    line = lines[i]

    device = torch.device('cpu')
    model, symbols, sample_rate, example_text, apply_tts = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                          model='silero_tts',
                                                                          language=language,
                                                                          speaker=f"{speaker}_16khz")
    model = model.to(device)  # gpu or cpu
    audio = apply_tts(texts=[line],
                      model=model,
                      sample_rate=sample_rate,
                      symbols=symbols,
                      device=device)

    wavs.append({"audio": audio[0].numpy()}) # audio[0] since their is only one line
  return wavs

def createFullAudio(rawAudioWavs, textLines, rate = 16000):
  full_audio = np.array([])

  pause_length = 1000
  pause = np.array([0] * pause_length)

  for i, obj in enumerate(rawAudioWavs):
      obj["time_length"] = (obj["audio"].shape[0] + pause_length) / rate # seconds
      obj["line"] = textLines[i]
      full_audio = np.concatenate((full_audio, pause, obj["audio"]))
  return full_audio


# Всяческие настройки

In [6]:
#@title Настройки GPT-Neo { run: "auto" }
#@markdown Используя `min_length` и `max_length` можно задать длинну видео по формуле
#@markdown $$\text{min_length} \cdot 2 / 190 \approx \text{длинна видео в минутах}$$
#@markdown `max_length` можно взять на 50 больше чем `min_length`.

# https://happytransformer.com/text-generation/settings/

min_length =  180#@param {type:"number"}
max_length =  230#@param {type:"number"}
do_sample = True #@param {type:"boolean"}
early_stopping = False #@param {type:"boolean"}
num_beams = 1 #@param {type:"number"}
temperature = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
top_k = 50 #@param {type:"slider", min:1, max:50, step:1}
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
no_repeat_ngram_size = 2 #@param {type:"slider", min:0, max:10, step:1}

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

# Генерация Шоу

## Текст

In [7]:
#@title Параметры текста

genre = "alone" #@param ['alone', 'america', 'angel', 'anger', 'animal', 'baby', 'beach', 'beautiful', 'beauty', 'believe', 'birth', 'brother', 'butterfly', 'car', 'carpe diem', 'change', 'chicago', 'childhood', 'children', 'christmas', 'cinderella', 'city', 'courage', 'crazy', 'culture', 'dance', 'dark', 'daughter', 'death', 'depression', 'despair', 'destiny', 'dream', 'evil', 'faith', 'family', 'father', 'fear', 'fire', 'food', 'football', 'freedom', 'friend', 'frog', 'funeral', 'funny', 'future', 'girl', 'god', 'graduation', 'greed', 'green', 'hair', 'happiness', 'happy', 'hate', 'heaven', 'hero', 'home', 'hope', 'house', 'hunting', 'husband', 'identity', 'innocence', 'january', 'joy', 'june', 'justice', 'kiss', 'laughter', 'life', 'lonely', 'loss', 'lost', 'love', 'lust', 'marriage', 'memory', 'mirror', 'money', 'moon', 'mother', 'murder', 'music', 'nature', 'night', 'ocean', 'paris', 'passion', 'peace', 'pink', 'poem', 'poetry', 'poverty', 'power', 'racism'] {type:"string"}
name =  "Why it's (sometimes) good to be bored" #@param{type:"string"}

name = name.lower()

# prompt = f"""
# Topic: {genre}
# Name: {name}
# Poem: """

prompt = name

In [8]:
# @title Генерация и форматирование текста
def generateText():
  return happy_gen.generate_text(prompt, args=gen_args).text

def formatText(text):
  newText = text.replace("<nl>", "\n").replace("\'", "'").replace(", ", ",\n")
  newText = re.sub("[^a-zA-Z,.\n\s]", "", newText)
  newText = newText.replace(".,", ".").replace(". \n", ".").replace(".\n", ".").replace(",\n", ".").replace(",", ".").replace("\n", ".")

  sentences = newText.split(".")
  lines = [line for line in sentences if len(line) > 2]
  return lines

rawGeneratedText = generateText()
textLines = formatText(rawGeneratedText) # Array of sentences

print(textLines)

['When all you care about is fame', 'Yes', 'Id be a dolt', 'if hed ever make me laugh', 'When I think I need money', 'or want to eat', 'steak', 'anything I do', 'then whats the point', 'Im too busy being rich', 'and now I want a mansion', 'Ill make my money out of nothing', 'when I can have a family', 'If my fathers in prison', 'this is where I belong', 'No matter how money lasts', 'it will never get better', 'If I make enough to live on', 'Just look at all those fat cats', 'on top of their diet', 'The ones who say I deserve death', 'And deserve to die for being fat', 'What about the ones that say that I should', 'Get more money than I give', 'For a house on the lake', ' They dont deserve', 'Life', 'but what about being free', ' Its', 'So much more', 'Than I could have had with me', 'without you', 'So the more you know', 'the happier you are', 'The more happiness youll find', 'Youll never make it out alive']


## Озвучка

In [9]:
# @title Генерация озвучки
import random
import numpy as np

speakers = ['lj']

def assignSpeakersToLines(texts):
  result = (speakers * len(texts))[:len(texts)]

#   for i in range(len(texts)):
#     result.append(random.choice(speakers))
  return result

speakerLines = assignSpeakersToLines(textLines)

rawAudioWavs = generateRawAudio(speakerLines, textLines)

full_audio = createFullAudio(rawAudioWavs, textLines)
displayAudio(full_audio)
createAudioFile(full_audio, "audio")

Output hidden; open in https://colab.research.google.com to view.

## Сохраняем для следующего колаба

In [10]:
#@markdown Создаем соотвествующий промпт для видео

#@markdown Запомните FPS!!!

fps = 12 #@param{type:"number"}
video_prompt = ""

dur = 0
for i, line in enumerate(rawAudioWavs):
    if i > 0:
        video_prompt += f",{dur}:({line['line']}:1| {rawAudioWavs[i-1]['line']}:0"

        if i != len(rawAudioWavs) - 1:
            video_prompt += f"| {rawAudioWavs[i+1]['line']}:0)"
        else:
            video_prompt += ")"
    else:
        video_prompt += f",{dur}:({line['line']}:1| {rawAudioWavs[i+1]['line']}:0.1)"

    dur += int(line["time_length"] * fps)

print(f"ещё запомните dur = {dur}")

ещё запомните dur = 621


In [16]:
#@markdown сохраняем промпт в файл

text_file = open(f"{working_dir}/video_prompt.txt", "w")
n = text_file.write(video_prompt)
text_file.close()

In [17]:
#@markdown сохраняем аудио файл

import os
import shutil

shutil.move("audio.wav", working_dir)

'/content/drive/Shareddrives/ML/AIIJC/working_dir/audio.wav'